In [1]:
#Methods

from typing import Final
from telegram import Update
from telegram.constants import ParseMode
from telegram.ext import Application, CommandHandler, MessageHandler, filters, ContextTypes

import codeforces_api
import random

import os
import json


#Commands
async def start_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text('Holi UwU')
    
    user_id = f"{context._user_id}"
    user_data_file = os.path.join('database', user_id)
    if(not os.path.isfile(user_data_file)):
        user_data = {'user_id':user_id,
                    'peak':0,
                    'cf_handle':'',
                    'menu':'register_menu'}
        write_dict_to_json(user_data, user_id)
        await update.message.reply_text('Hey you are not registered. Give me your codeforces handle you bastard!!!')


async def help_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    #await update.message.reply_text('Generic help message')
    await update.message.reply_text(text="<a href='codeforces.com'>Codeforces</a>", parse_mode=ParseMode.HTML)


async def random_problems_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    user_id = f"{context._user_id}"
    user_data = read_json_to_dict(user_id)
    user_handle = user_data['cf_handle']
    user_rating = get_user_rating(user_handle)

    random_problems = get_random_problems(user_rating, user_rating + 300, [], [user_handle], 3)
    for link in random_problems:
        await update.message.reply_text(link)



#Handle Responses
def handle_private_response(text: str, user_data) -> str:    
    user_menu = user_data['menu']

    if(user_menu == 'register_menu'):  #REGISTER MENU
        cf_account = text

        if(cf_user_exists(cf_account)):
            user_data['cf_handle'] = cf_account
            user_data['menu'] = 'main_menu'
            write_dict_to_json(user_data, user_data['user_id'])

            return 'Ok, you have been successfully registered UwU :)'
        else:
            return 'Liar!!! Thats not even a codeforces handle. Try again'
        
    return 'What the hell are u saying bro'




async def handle_message(update: Update, context: ContextTypes.DEFAULT_TYPE):
    message_type: str = update.message.chat.type
    text: str = update.message.text

    print(f'User {update.message.chat.username} in {message_type}: "{text}"')

    if message_type == 'group' or message_type == 'supergroup':
        await update.message.reply_text('Who knows...')
    else:
        user_id = f"{context._user_id}"
        user_data = read_json_to_dict(user_id)
        await update.message.reply_text(handle_private_response(text, user_data))
    


async def error(update: Update, context: ContextTypes.DEFAULT_TYPE):
    print(f'Update {update} caused error {context.error}')



#Codeforces API methods
def get_user_rating(handle):
    cf_api = codeforces_api.CodeforcesApi()
    user = cf_api.user_info(handles=[handle])

    if user:
        return user[0].rating
    else:
        return None


def cf_user_exists(handle):
    cf_api = codeforces_api.CodeforcesApi()

    try:
        user = cf_api.user_info(handles=[handle])
        return True if user else False
    except:
        return False

def search_problems(min_rating, max_rating, tags, not_made_by):
    cf_api = codeforces_api.CodeforcesApi()
    problems = cf_api.problemset_problems()['problems']

    # Filter problems based on rating
    problems = [problem for problem in problems if problem.rating is not None and min_rating <= problem.rating <= max_rating]

    # Filter problems based on tags
    if tags:
        problems = [problem for problem in problems if any(tag in problem.tags for tag in tags)]

    # Filter out problems solved by users in not_made_by
    for handle in not_made_by:
        user_submissions = cf_api.user_status(handle=handle)
        solved_problems = {submission.problem.name for submission in user_submissions if submission.verdict == 'OK'}
        problems = [problem for problem in problems if problem.name not in solved_problems]

    # Get the links of the problems 
    links = []
    for problem in problems:
        links.append(get_problem_link(problem)) 

    # Return list of problem links
    return links #['http://codeforces.com/problemset/problem/{}/{}'.format(problem.contestId, problem.index) for problem in problems]


#Others
def get_random_problems(min_rating, max_rating, tags, not_made_by, amount):
    
    available_problems = search_problems(min_rating, max_rating, tags, not_made_by)

    random_problems = random.sample(available_problems, amount)

    return random_problems


def get_problem_link(problem):
    return f"https://codeforces.com/problemset/problem/{problem.contest_id}/{problem.index}"


def read_token(filename):
    
    #(task) Launch error if something is missing

    with open(os.path.join('tokens', filename), 'r') as file:
        content = file.read()
    return content


def write_dict_to_json(dict, filename):
    # Ensure the 'database' directory exists
    if not os.path.exists('database'):
        os.makedirs('database')

    # Write the dictionary to a JSON file in the 'database' directory
    with open(os.path.join('database', filename), 'w') as file:
        json.dump(dict, file)


def read_json_to_dict(filename):
    
    #(task) Launch error if something is missing

    with open(os.path.join('database', filename), 'r') as file:
        my_dict = json.load(file)
    return my_dict

In [ ]:
probs = get_random_problems(0, 1400, [], [], 3)
probs

In [2]:
bot_username = '@cfquestboardbot'
bot_token = read_token('telegram_bot_token') 
cf_api_key = read_token('codeforces_api_key')
cf_api_secret = read_token('codeforces_api_secret')


if __name__ == '__main__':
    
    print('Starting...')
    app = Application.builder().token(bot_token).build()

    #Commands
    app.add_handler(CommandHandler('start', start_command))
    app.add_handler(CommandHandler('help', help_command))
    app.add_handler(CommandHandler('random_problems', random_problems_command))

    #Messages
    app.add_handler(MessageHandler(filters.TEXT, handle_message))

    #Errors
    app.add_error_handler(error)

    #Polls the bot
    print('Polling...')
    await app.initialize() # inits bot, update, persistence
    await app.start()
    await app.updater.start_polling(poll_interval=1)

print('All Good')

Starting...
Polling...
All Good


In [4]:
#pip install codeforces-api
#update.message.reply_text(text="<a href='https://www.google.com/'>Google</a>", parse_mode=ParseMode.HTML)   to send links